In [1]:
import ee
import geemap
from geemap import ml
from sklearn import ensemble
import pandas as pd


# Inicializa la autenticación y la inicialización de Google Earth Engine
ee.Authenticate()
ee.Initialize(project='ee-facuboladerasgee')

In [38]:
# roi = ee.FeatureCollection('projects/ee-facuboladerasgee/assets/Forestaciones_uru')
Map = geemap.Map(center=[0, 0], zoom=2)
Map.add_basemap('SATELLITE')
Map

year = 2024

start = f'{year}-01-01'
end = f'{year}-12-31'
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [39]:
# Capturar el polígono dibujado como ROI
roi = Map.user_roi
coords = roi.coordinates()

print(coords.getInfo())

geom = ee.Geometry.Polygon(coords)
roi = geom

[[[-58.938904, -34.354774], [-58.938904, -33.991196], [-58.452759, -33.991196], [-58.452759, -34.354774], [-58.938904, -34.354774]]]


In [71]:
s2_col = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterBounds(roi) \
    .filterDate(start, end) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))

cloud_col = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY') \
    .filterBounds(roi) \
    .filterDate(start, end)

def mask_clouds(image):
    cloud_score = ee.Image(cloud_col.filterMetadata('system:index', 'equals', image.get('system:index')).first())
    mask = cloud_score.select('probability').lt(10)
    return image.updateMask(mask)

# Apply cloud masking and compute median
s2_masked = s2_col.map(mask_clouds)
s2_image = s2_masked.median().toFloat().clip(roi)


# Calculate indices (NDVI, MNDWI, NDBI, EVI, SAVI) for Sentinel-2
ndvi = s2_image.normalizedDifference(['B8', 'B4']).rename('NDVI')
mndwi = s2_image.normalizedDifference(['B3', 'B11']).rename('MNDWI')
ndbi = s2_image.normalizedDifference(['B11', 'B8']).rename('NDBI')

evi = s2_image.expression(
    '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))',
    {
        'NIR': s2_image.select('B8'),
        'RED': s2_image.select('B4'),
        'BLUE': s2_image.select('B2')
    }
).rename('EVI')

savi = s2_image.expression(
    '((NIR - RED) / (NIR + RED + 0.5)) * 1.5',
    {
        'NIR': s2_image.select('B8'),
        'RED': s2_image.select('B4')
    }
).rename('SAVI')


s2_image = s2_image.addBands([ndvi, mndwi, ndbi, evi, savi])

# Add Dynamic World label to the image
dw_col = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1') \
    .filterBounds(roi) \
    .filterDate(start, end)

dw_label = dw_col.select('label').median().clip(roi)
s2_image = s2_image.addBands(dw_label.rename('label'))

# Incorporate DEM and slope
dem = ee.Image('USGS/SRTMGL1_003').select('elevation').clip(roi)
slope = ee.Terrain.slope(dem).rename('slope').clip(roi)
aspect = ee.Terrain.aspect(dem).rename('aspect').clip(roi)

In [72]:
# -------------------------
# Sentinel-1 minimal improvements (same flow / same outputs)
# -------------------------

def mask_edge(image):
    # image is single-band (VV or VH) because of .select(polarization)
    edge = image.lt(-30.0)  # dB threshold for border noise
    masked = image.mask().And(edge.Not())
    return image.updateMask(masked)

def create_filtered_collection(polarization):
    return (ee.ImageCollection('COPERNICUS/S1_GRD')
            .filterBounds(roi)
            .filterDate(start, end)
            .filter(ee.Filter.eq('instrumentMode', 'IW'))
            .filter(ee.Filter.listContains('transmitterReceiverPolarisation', polarization))
            .select(polarization)
            .map(mask_edge))

img_vv = create_filtered_collection('VV')
img_vh = create_filtered_collection('VH')

# Compute medians for Sentinel-1
s1_vv = img_vv.median().rename('VV').clip(roi)
s1_vh = img_vh.median().rename('VH').clip(roi)

# Combine Sentinel-1 bands
s1_combined = ee.Image.cat([s1_vv, s1_vh])

# Cache selections (avoids repeated selects)
vv = s1_combined.select('VV')
vh = s1_combined.select('VH')

# Safe denominator for RVI to avoid division by 0
den = vv.add(vh).max(1e-6)

# Calculate RVI, VV+VH, and VV-VH (same band names)
rvi = vh.multiply(4).divide(den).rename('RVI')

vv_plus_vh = vv.add(vh).rename('VV_plus_VH')
vv_minus_vh = vv.subtract(vh).rename('VV_minus_VH')

# Add bands to the image
s1_image = s1_combined.addBands([rvi, vv_plus_vh, vv_minus_vh])

# Combine all layers into the final image
image = s2_image.addBands([
    s1_image,
    dem.rename('elevation'),
    slope,
    aspect
])

In [73]:
def calculate_texture_metrics(image, index_name, roi, scale_factor=100, size=3):
    
    index_norm = image.select(index_name).multiply(scale_factor).add(scale_factor).toInt32().clip(roi)
    
    glcm = index_norm.glcmTexture(size=size)
    contrast = glcm.select(f'{index_name}_contrast').rename(f'{index_name}_Contrast')
    correlation = glcm.select(f'{index_name}_corr').rename(f'{index_name}_Correlation')
    entropy = glcm.select(f'{index_name}_ent').rename(f'{index_name}_Entropy')
    inertia = glcm.select(f'{index_name}_inertia').rename(f'{index_name}_Inertia')
    
    # Agregar las bandas de textura a la imagen original
    return image.addBands([contrast, correlation, entropy, inertia])


image = calculate_texture_metrics(image, 'NDVI', roi)
image = calculate_texture_metrics(image, 'SAVI', roi)

bands = [

    # --- Sentinel-2 ---
    'B11', 'B12', 'B8', 'B4', 'B3', 'B2',
    # --- Índices ---
    'NDVI', 'SAVI', 'EVI', 'MNDWI', 'NDBI',

    # --- Texturas NDVI & SAVI ---
    'NDVI_Contrast', 'NDVI_Correlation', 'NDVI_Entropy', 'NDVI_Inertia',
    'SAVI_Contrast', 'SAVI_Correlation', 'SAVI_Entropy', 'SAVI_Inertia',

    # --- Sentinel-1 (ARD) ---
    'VV',
    'VH',    
    'RVI'    ,
    'VV_plus_VH',
    'VV_minus_VH',

    # --- Topografía ---
    'elevation',
    'slope',
    'aspect',

    # --- Label ---
    'label'
]


image = image.select(bands)

In [74]:
Map.centerObject(roi, 10)
Map.addLayer(image.select('VV'), {}, 'VV', False)
Map.addLayer(image.select('B3'), {}, 'B3', False)
Map.addLayer(image.select('elevation'), {}, 'elevation', )
Map

Map(bottom=157878.0, center=[-34.17309407837407, -58.69583149999968], controls=(WidgetControl(options=['positi…

In [75]:
def mask_excluding_built(image, built_id=6):
    dw = (ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1')
          .filterBounds(roi)
          .filterDate(start, end)
          .select('label')
          .mode()      # clave para clases
          .clip(roi))

    not_built_mask = dw.neq(built_id)  # <-- TODO menos built
    return image.updateMask(not_built_mask)


# Obtener los datos GEDI de la altura de dosel
def get_gedi_canopy_height(band_name):
    return (ee.ImageCollection('LARSE/GEDI/GEDI02_A_002_MONTHLY')
        .filterBounds(roi)
        .filterDate(start, end)
        .map(lambda image: image.updateMask(
            image.select('degrade_flag').eq(0)))
        .select(band_name)
        .median()
        .toFloat()
        .clip(roi)
    )

gedi_rh98 = get_gedi_canopy_height('rh98').rename('rh98')

# excluir construido
gedi_masked = mask_excluding_built(gedi_rh98, built_id=6)

image = image.addBands(gedi_masked.rename('rh98'))


In [76]:
Map.centerObject(roi, 10)
Map.addLayer(image.select('rh98'), {}, 'rh95', )
Map

Map(bottom=157878.0, center=[-34.17309407837407, -58.69583149999968], controls=(WidgetControl(options=['positi…

In [77]:
import time

sample = image.sample(
    scale=10,  # Ajusta la escala según sea necesario para tus datos
    region=roi,
    geometries=True  # Incluir geometría de los puntos
)

# Exportar la tabla de muestra a Google Drive en formato CSV
export_task = ee.batch.Export.table.toDrive(
    collection=sample,
    description='sample_delta_CH_S1-S2',
    folder='CH-delta',
    fileNamePrefix=f'Datos_RF_2021-chm_trees',
    fileFormat='CSV'
)


# Iniciar la tarea de exportación
export_task.start()

    # Esperar a que la tarea de exportación se complete
export_task.status()

    # Verificar el estado de la tarea y mostrar un mensaje de éxito
while export_task.active():
    print('Exportación en progreso...')
    time.sleep(30)  # Esperar 30 segundos antes de verificar el estado nuevamente
    
    if export_task.status()['state'] == 'COMPLETED':
        print(f'Exportación completada con éxito.')
    else:
        print(f'Error en la exportación: {export_task.status()}')

Exportación en progreso...
Error en la exportación: {'state': 'RUNNING', 'description': 'sample_delta_CH_S1-S2', 'priority': 100, 'creation_timestamp_ms': 1763731445618, 'update_timestamp_ms': 1763731476642, 'start_timestamp_ms': 1763731456051, 'task_type': 'EXPORT_FEATURES', 'attempt': 1, 'batch_eecu_usage_seconds': 0.157208681, 'id': 'FHIN57JMI76RZAKY6XB7TTGE', 'name': 'projects/ee-facuboladerasgee/operations/FHIN57JMI76RZAKY6XB7TTGE'}
Exportación en progreso...
Error en la exportación: {'state': 'RUNNING', 'description': 'sample_delta_CH_S1-S2', 'priority': 100, 'creation_timestamp_ms': 1763731445618, 'update_timestamp_ms': 1763731506649, 'start_timestamp_ms': 1763731456051, 'task_type': 'EXPORT_FEATURES', 'attempt': 1, 'batch_eecu_usage_seconds': 800.02947998, 'id': 'FHIN57JMI76RZAKY6XB7TTGE', 'name': 'projects/ee-facuboladerasgee/operations/FHIN57JMI76RZAKY6XB7TTGE'}
Exportación en progreso...
Error en la exportación: {'state': 'RUNNING', 'description': 'sample_delta_CH_S1-S2', 'p